In [114]:
import pandas as pd

In [115]:
data = pd.read_csv("../../data/rgs_app_2025-03-04.csv", dtype_backend="numpy_nullable")

In [116]:
data = data.dropna(subset="PRESCRIPTION_ID")

In [117]:
filtered = data.sort_values(by=["PATIENT_ID", "PROTOCOL_ID", "SESSION_ID"]).iloc[:,[0,15]+list(range(16, 20))+[26]]

In [118]:
patient_775 = filtered[filtered["PATIENT_ID"] == 775]

In [119]:
ps = patient_775["PRESCRIPTION_ID"].unique().tolist()

In [120]:
patient_775[patient_775["PRESCRIPTION_ID"] == ps[0]]["PRESCRIPTION_ENDING_DATE"].unique().__len__()

1

In [121]:
filtered

,PATIENT_ID,PRESCRIPTION_ID,SESSION_ID,PROTOCOL_ID,PRESCRIPTION_STARTING_DATE,PRESCRIPTION_ENDING_DATE,WEEKDAY
8,204,136298.0,43964.0,205.0,2024-09-10 10:43:43,2100-01-01 00:00:00,TUESDAY
9,204,136298.0,43976.0,205.0,2024-09-10 10:43:43,2100-01-01 00:00:00,TUESDAY
10,204,136298.0,44072.0,205.0,2024-09-10 10:43:43,2100-01-01 00:00:00,TUESDAY
16,204,136299.0,44096.0,205.0,2024-09-10 10:43:43,2100-01-01 00:00:00,WEDNESDAY
17,204,136299.0,44117.0,205.0,2024-09-10 10:43:43,2100-01-01 00:00:00,WEDNESDAY
...,...,...,...,...,...,...,...
5563,3318,147955.0,55336.0,225.0,2024-11-18 15:04:59,2100-01-01 00:00:00,SUNDAY
5568,3318,147949.0,55350.0,225.0,2024-11-18 15:04:59,2100-01-01 00:00:00,MONDAY
5572,3318,147950.0,55362.0,225.0,2024-11-18 15:04:59,2100-01-01 00:00:00,TUESDAY
5576,3318,147951.0,55371.0,225.0,2024-11-18 15:04:59,2100-01-01 00:00:00,WEDNESDAY


In [123]:
# Check if all prescriptions ending are the same
cl = filtered["PATIENT_ID"].unique().tolist()
for c in cl:
    df = filtered[filtered["PATIENT_ID"] == c]
    ps = df["PRESCRIPTION_ID"].unique().tolist()
    for p in ps:
        assert df[df["PRESCRIPTION_ID"] == p]["PRESCRIPTION_ENDING_DATE"].unique().__len__() == 1, f"Prescription {p} of patient {c} has different ending dates"

In [106]:
class Session:
    def __init__(self, df):
        self._df = df

    def __getattr__(self, name):
        attr = getattr(self._df, name)
        # Wrap return value only if it's a DataFrame
        if callable(attr):
            def method(*args, **kwargs):
                result = attr(*args, **kwargs)
                if isinstance(result, pd.DataFrame):
                    return Session(result)
                return result
            return method
        else:
            return attr

    def __repr__(self):
        return repr(self._df)

    def get(self, id):
        return self._df[self._df["PATIENT_ID"] == id]
